!["RESIM"](flo_veri_hikayesi.png)

## Veri Setini Okuma ve Veri Seti Hakkında Bilgi Edinme

- Gerekli kütüphanelerin import edilmesi

In [36]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
#!pip install Lifetimes
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [37]:
df =pd.read_csv("flo_data_20k.csv")

In [38]:
df_=df.copy()

In [39]:
def data_preparation(df):
    print("################################ VERİ ÖN HAZIRLIK SÜRECİ ####################################","\n",
      "1-İlk 10 Gözlem:","\n\n",df.head(10),"\n\n",
      "2-Değişken İsimleri:","\n\n",df.columns,"\n\n",
      "3-Betimsel İstatistik:","\n\n",df.describe().T,"\n\n",
      "4:Boş Gözlem Sayısı:","\n\n",df.isnull().sum(),"\n\n",
      "5-Değişkenlerin Türü:","\n\n",df.dtypes,"\n\n")

In [40]:
data_preparation(df)

################################ VERİ ÖN HAZIRLIK SÜRECİ #################################### 
 1-İlk 10 Gözlem: 

                               master_id order_channel last_order_channel  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile   
2  69b69676-1a40-11ea-941b-000d3a38a36f   Android App        Android App   
3  1854e56c-491f-11eb-806e-000d3a38a36f   Android App        Android App   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       Desktop            Desktop   
5  e585280e-aae1-11e9-a2fc-000d3a38a36f       Desktop            Offline   
6  c445e4ee-6242-11ea-9d1a-000d3a38a36f   Android App        Android App   
7  3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f        Mobile            Offline   
8  cfbda69e-5b4f-11ea-aca7-000d3a38a36f   Android App        Android App   
9  1143f032-440d-11ea-8b43-000d3a38a36f        Mobile             Mobile   

  first_order_date last_order_date last_order_d

Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlamış bulunmaktayız.
Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.Bu nedenle alt ve üst limitlerini round() ile yuvarlıyoruz.

In [41]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = (quartile3 + 1.5 * interquantile_range)
    low_limit =(quartile1 - 1.5 * interquantile_range)
    return low_limit, up_limit

In [42]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit.round()
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit.round()


In [43]:
replace_with_thresholds(df,"order_num_total_ever_online")
replace_with_thresholds(df,"order_num_total_ever_offline")#Değişkenlerin varsa aykırı değerlerini baskılama
replace_with_thresholds(df,"customer_value_total_ever_online",)
replace_with_thresholds(df,"customer_value_total_ever_offline",)

 Omnichannel müşterilerin hem online'dan hem de offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam
alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz

In [44]:
df["total_transaction"]= df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["total_transaction"].head()

0     5.0
1    21.0
2     5.0
3     2.0
4     2.0
Name: total_transaction, dtype: float64

In [45]:
df["total_price"]= df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
df["total_price"].head()

0     939.37
1    2013.55
2     585.32
3     121.97
4     209.98
Name: total_price, dtype: float64

Değişken tipleri incelendiğinde Tarih date eden değişkenlerin tipinin object olduğu gözükmektedir. Bu yüzden date ifade eden değişkenlerin tipini date'e çeviriyoruz.

In [46]:
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

## CLTV Veri Yapısının Oluşturulması

Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak belirliyoruz.

In [48]:
today_date = df["last_order_date"].max() + dt.timedelta(days=2)
today_date

Timestamp('2021-06-01 00:00:00')

 Customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı yeni bir cltv dataframe'i oluşturuyoruz.
Monetary değeri satın alma başına ortalama değer olarak, recency ve tenure değerleri ise haftalık cinsten ifade ediyoruz.

In [49]:
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"] - df["first_order_date"]).dt.days) / 7
cltv_df["T_weekly"] = ((today_date - df["first_order_date"]).astype('timedelta64[D]')) / 7
cltv_df["frequency"] = df["total_transaction"]
cltv_df["monetary_cltv_avg"] = df["total_price"] / df["total_transaction"]
cltv_df 

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.0,187.874000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.0,95.883333
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.0,117.064000
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.0,60.985000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.0,104.990000
...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,41.142857,88.428571,3.0,133.986667
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,42.285714,65.285714,2.0,195.235000
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,88.714286,89.857143,3.0,210.980000
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,98.428571,113.857143,6.0,168.295000


# BG/NBD, Gamma-Gamma Modellerinin Kurulması ve CLTV’nin Hesaplanması

### BG/NBD modelini fit ediyoruz.
- 3 ay içerisinde müşterilerden beklenen satın almaları tahmin ederek ve exp_sales_3_month olarak cltv
dataframe'ine ekliyoruz.
- 6 ay içerisinde müşterilerden beklenen satın almaları tahmin ederek ve exp_sales_6_month olarak cltv
dataframe'ine ekliyoruz.

In [50]:
bgf = BetaGeoFitter(penalizer_coef=0.001)


In [51]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [52]:
cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                           cltv_df['frequency'],
                                           cltv_df['recency_cltv_weekly'],
                                           cltv_df['T_weekly'])

In [53]:
cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                           cltv_df['frequency'],
                                           cltv_df['recency_cltv_weekly'],
                                           cltv_df['T_weekly'])

### Gamma-Gamma modelini fit ediyoruz.  
 Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv
dataframe'ine ekliyoruz.

plot_period_transactions(bgf)
plt.show()

In [54]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary_cltv_avg']).head(10)



0    193.632679
1     96.665048
2    120.967619
3     67.320145
4    114.325108
5     71.348148
6     98.133357
7     89.570194
8    217.300903
9     90.814616
dtype: float64

Adım3: 6 aylık CLTV hesaplayarak  cltv ismiyle dataframe'e ekledik.
    • Cltv değeri en yüksek 20 kişiyi gözlemlemekteyiz.

In [55]:
cltv_df["Cltv"] = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'],
                                       cltv_df['monetary_cltv_avg'],
                                       time=6, 
                                       freq="W", 
                                       discount_rate=0.01)


In [56]:
cltv_df.sort_values("Cltv",ascending = False).head(20)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,Cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.857143,7.857143,4.0,1401.800000,1.094385,2.188769,3327.777040
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.142857,13.142857,11.0,758.085455,1.970108,3.940216,3172.394390
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.714286,101.000000,7.0,1106.467143,0.722238,1.444476,1708.982063
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.285714,74.571429,16.0,501.873750,1.565309,3.130618,1662.613492
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714286,67.285714,52.0,166.224615,4.656138,9.312276,1628.887381
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.428571,34.428571,8.0,601.226250,1.265456,2.530912,1623.812684
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.285714,9.428571,2.0,862.690000,0.793924,1.587847,1538.855906
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.714286,13.000000,17.0,259.865294,2.780689,5.561378,1529.227995
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571429,58.714286,31.0,228.530000,3.083779,6.167558,1485.819156
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.857143,15.571429,3.0,743.586667,0.871564,1.743128,1422.999674


**Adım1:** 6 aylık CLTV'ye göre tüm müşterileri 4 gruba (segmente) ayırma işlemi yaparak  grup isimlerini veri setine eklemiş bulunmaktayız.

In [59]:
cltv_df["segment"] = pd.qcut(cltv_df["Cltv"], 4, labels=["D", "C", "B", "A"])


In [60]:
cltv_df.head(5)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,Cltv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.0,187.874000,0.973927,1.947853,395.733234,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.0,95.883333,0.983161,1.966323,199.430693,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.0,117.064000,0.670586,1.341172,170.224184,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.0,60.985000,0.700412,1.400824,98.945526,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.0,104.990000,0.396039,0.792077,95.011659,D


In [61]:
cltv_df["segment"].value_counts()

D    4987
C    4986
B    4986
A    4986
Name: segment, dtype: int64